In [2]:
%load_ext sql


In [3]:
%sql sqlite:///vivino.db

In [4]:
%%sql
SELECT * FROM wines LIMIT 5;

 * sqlite:///vivino.db
Done.


id,name,is_natural,region_id,winery_id,ratings_average,ratings_count,url,acidity,fizziness,intensity,sweetness,tannin,user_structure_count
1213,Vin Santo di Montepulciano,0,2693,1574,4.6,773,https://www.seulementproduitsitaliens.be/spec_collezioni.php?CODICEWT=10AVVS92H,4.492628,None,3.9921334,4.952543,None,0
1268,St. Henri Shiraz,0,7,1305,4.4,17326,https://www.wijnig.nl/penfolds-st-henri-shiraz-2018.html,3.0152445,None,4.612774,2.092192,3.1030886,1309
1471,Siepi,0,394,1620,4.4,8129,https://www.topwijnen.be/nl/product/wijnen/italie/toscane/igt-toscane/wn6010264/wn6010264,3.156144,None,3.4918664,1.807986,2.9856336,755
1647,Solare Toscana,0,394,4772,4.2,4961,https://www.seulementproduitsitaliens.be/spec_collezioni.php?CODICE=2001658,3.1583757,None,3.3216164,1.7018968,3.0380645,485
1651,Tenuta Tignanello 'Solaia',0,394,1252,4.6,33143,http://www.vivino.com/wines/159347388,3.0641637,None,3.4507132,1.8885487,2.9705827,2131


### 1: 10 Wines to increase sales
I chose the wines based on high average ratings and a significant number of ratings
High average ratings generally indicate customer satisfaction and the quality of the wine. It is a common assumption that products with higher ratings are more likely to attract new customers and retain existing ones.

In [5]:
%%sql
SELECT id, name, ratings_average, ratings_count
FROM wines
ORDER BY ratings_average DESC, ratings_count DESC
LIMIT 10;

 * sqlite:///vivino.db
Done.


id,name,ratings_average,ratings_count
1611255,Cabernet Sauvignon,4.8,2941
1187886,Amarone della Valpolicella Classico Riserva,4.8,587
6534388,Fratini Bolgheri Superiore,4.8,153
5806244,Mágico,4.8,146
7266631,Cristal Rosé Vinothèque,4.8,88
9330771,Batard-Montrachet Grand Cru,4.8,34
77137,Unico,4.7,45140
1153863,Sauternes,4.7,44126
66294,Special Selection Cabernet Sauvignon,4.7,41236
1166837,Pomerol,4.7,32157


### 2: Prioritize a country based on marketing budget:
Assuming that a lower number of wineries means less marketing expense, prioritized countries with fewer wineries, which might correlate to lower competition and marketing expense so each winery might be more recognizable. This could potentially reduce marketing costs as each marketing action (like an ad or an event) could reach a relatively larger percentage of the country's wine market. It could also mean that the wine market is less saturated in that country, allowing for easier penetration and brand recognition.

In [8]:
%%sql
SELECT code, name
FROM countries
ORDER BY wineries_count ASC
LIMIT 1;

 * sqlite:///vivino.db
Done.


code,name
md,Moldavie


### 3: Awards to the best wineries:
Chose wineries based on the highest average rating of their wines.
Attention, due to known issues with the database, this query might not provide a useful answer because there's a mismatch between winery names and wine names.

In [9]:
%%sql
SELECT winery_id, wineries.name, AVG(ratings_average) as avg_rating
FROM wines
JOIN wineries ON wineries.id = wines.winery_id
GROUP BY winery_id
ORDER BY avg_rating DESC
LIMIT 3;

 * sqlite:///vivino.db
Done.


winery_id,name,avg_rating
75712,Corte di Cama Sforzato di Valtellina,4.5
1651,Tenuta Tignanello 'Solaia',4.5
1652,Tignanello,4.4


### 4: Find wines with specific taste keywords:
Finding wines matching all 5 of the keywords and having at least 10 user confirmations.
This query might be uncertain if the count field does not accurately reflect the number of user confirmations for each keyword.
Name suggestion: "Orchard Mornings"

In [36]:
%%sql
SELECT w.id, w.name, COUNT(*) as user_confirmed
FROM wines w
JOIN keywords_wine kw ON w.id = kw.wine_id
JOIN keywords k ON k.id = kw.keyword_id
WHERE k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus') AND kw.count >= 10
GROUP BY w.id, w.name
HAVING COUNT(DISTINCT k.name) >= 5
ORDER BY user_confirmed DESC;

 * sqlite:///vivino.db
Done.


id,name,user_confirmed
18931,La Grande Année Brut Champagne,7
74304,Cristal Brut Champagne (Millésimé),7
79162,Belle Epoque Brut Champagne,7
79631,Vintage,7
79836,La Grande Dame Brut Champagne,7
86684,Brut Champagne,7
1105696,Trebbiano d'Abruzzo,7
1127349,Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne,7
1135215,Sauternes (Premier Grand Cru Classé),7
1136950,Comtes de Champagne Blanc de Blancs,7


### 5: Select wines that are easy to find worldwide:
Finding the top 3 most common grapes and the best 5 wines for each
This query might not provide a useful answer as the exact grape for an individual wine is not always known.

In [15]:
%%sql
WITH CommonGrapes AS (
SELECT grape_id
FROM most_used_grapes_per_country
ORDER BY wines_count DESC
LIMIT 3
)
SELECT g.name AS grape_name, w.name, v.ratings_average
FROM vintages v
JOIN wines w ON v.wine_id = w.id
JOIN CommonGrapes cg ON w.id = cg.grape_id
ORDER BY v.ratings_average DESC
LIMIT 5;

 * sqlite:///vivino.db
(sqlite3.OperationalError) no such column: g.name
[SQL: WITH CommonGrapes AS (
SELECT grape_id
FROM most_used_grapes_per_country
ORDER BY wines_count DESC
LIMIT 3
)
SELECT g.name AS grape_name, w.name, v.ratings_average
FROM vintages v
JOIN wines w ON v.wine_id = w.id
JOIN CommonGrapes cg ON w.id = cg.grape_id
ORDER BY v.ratings_average DESC
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 6: Create a country leaderboard:
Average wine rating for each country

In [17]:
%%sql
SELECT c.name, AVG(w.ratings_average) as avg_rating
FROM wines w
JOIN regions r ON w.region_id = r.id
JOIN countries c ON r.country_code = c.code
GROUP BY c.code
ORDER BY avg_rating DESC;

 * sqlite:///vivino.db
Done.


name,avg_rating
Allemagne,4.500000000000001
Israël,4.5
États-Unis,4.4905405405405405
Moldavie,4.4799999999999995
Hongrie,4.472727272727273
Afrique du Sud,4.459090909090909
Australie,4.458333333333333
France,4.447129909365562
Espagne,4.4436170212765935
Portugal,4.435714285714287


In [34]:
# Visualizing the results:
import plotly.express as px
import pandas as pd

data = {
    'country': ['Allemagne', 'Israël', 'États-Unis', 'Moldavie', 'Hongrie', 'Afrique du Sud', 
             'Australie', 'France', 'Espagne', 'Portugal', 'Chili', 'Italie', 
             'Argentine', 'Roumanie', 'Grèce', 'Suisse', 'Croatie'],
    'avg_rating': [4.500000000000001, 4.5, 4.4905405405405405, 4.4799999999999995, 4.472727272727273, 
                   4.459090909090909, 4.458333333333333, 4.447129909365562, 4.4436170212765935, 
                   4.435714285714287, 4.43125, 4.430026109660581, 4.417391304347826, 4.4, 4.4, 4.35, 4.3]
}

name_map = {
    'Allemagne': 'Germany',
    'Israël': 'Israel',
    'États-Unis': 'United States',
    'Moldavie': 'Moldova',
    'Hongrie': 'Hungary',
    'Afrique du Sud': 'South Africa',
    'Australie': 'Australia',
    'France': 'France',
    'Espagne': 'Spain',
    'Portugal': 'Portugal',
    'Chili': 'Chile',
    'Italie': 'Italy',
    'Argentine': 'Argentina',
    'Roumanie': 'Romania',
    'Grèce': 'Greece',
    'Suisse': 'Switzerland',
    'Croatie': 'Croatia'
}

df = pd.DataFrame(data)
df['country'] = df['country'].map(name_map)

fig = px.bar(df, x='country', y='avg_rating', title='Average Wine Rating by Country',
             labels={'country': 'Country Code', 'avg_rating': 'Average Rating'},
             color='avg_rating', text='avg_rating')
fig.show()

### Wine recommendations for a VIP client who likes Cabernet Sauvignon:
Assuming Cabernet Sauvignon is mentioned in wine names, selecting top 5 wines

In [26]:
%%sql
SELECT id, name, ratings_average, ratings_count
FROM wines
WHERE name LIKE '%Cabernet Sauvignon%'
ORDER BY ratings_average DESC, ratings_count DESC
LIMIT 10;


 * sqlite:///vivino.db
Done.


id,name,ratings_average,ratings_count
1611255,Cabernet Sauvignon,4.8,2941
66294,Special Selection Cabernet Sauvignon,4.7,41236
66284,Cabernet Sauvignon,4.6,157944
3879,Cabernet Sauvignon (Signature),4.6,13730
2653222,CASK 23 Cabernet Sauvignon,4.6,9464
1145391,Cabernet Sauvignon,4.6,5927
3075039,Au Paradis Cabernet Sauvignon,4.6,3004
87360,Beckstoffer To Kalon Vineyard Cabernet Sauvignon,4.6,1557
94692,Cabernet Sauvignon Coeur De Vallée,4.6,544
2816852,Cabernet Sauvignon,4.6,152
